In [22]:
import re
import os
import ast
import pickle
import pandas as pd

from google import genai

from utils import *

from dotenv import load_dotenv


pd.set_option('display.max_columns', None)

load_dotenv() 


True

In [23]:
gemini_key = os.getenv("GEMINI_API_KEY")


In [3]:
tests = [
    # email + teléfono + nombre
    "Contactar a Juan Perez al +57 300-123-4567 o jperez@mail.com para validar.",
    # dirección + nombre
    "La propiedad está en 135 Abbott St y el responsable es Maria Lopez.",
    # caso que quede vacío
    "Juan Perez, +57 300-123-4567, jperez@mail.com, 22 Main Street"
]

res = [sanitizar_texto(txt) for txt in tests]

# Salida estructurada

In [4]:
casos_prueba = [
    {
        "id_propiedad": "077-0044-0000",
        "nivel_alerta": "ALTA",
        "motivo": "Inconsistencia detectada. Llamar a Maria Lopez al (301) 555-0199 para confirmar datos.",
        "recomendacion": "Verificar en Calle 123 #45-67 y escribir a mlopez@ejemplo.com para soporte.",
    },
    {
        "id_propiedad": "030-0452-0000",
        "nivel_alerta": "MEDIA",
        "motivo": "Posible error de clasificación. Contactar a Carlos Perez al +57 310-555-0000.",
        "recomendacion": "Solicitar documentación adicional al correo carlos.perez@correo.co y ajustar registro si aplica.",
    },
    {
        "id_propiedad": "015-0467-0000",
        "nivel_alerta": "BAJA",
        "motivo": "Caso atípico por ubicación. Responsable: Ana Gomez, teléfono 3001234567.",
        "recomendacion": "Revisar historial de pagos y enviar notificación a ana.gomez@mail.com con pasos a seguir.",
    },
    {
        "id_propiedad":"077-0144-0000",
        "nivel_alerta": "CRITICA",
        "motivo":"Juan Perez, +57 300-123-4567, jperez@mail.com, Calle 123 #45-67",
        "recomendacion":"Calle 60N #12C-08"
    }
]

for caso in casos_prueba:
    print(salida_estructurada(caso))

{'id_propiedad': '077-0044-0000', 'nivel_alerta': 'ALTA', 'motivo_tecnico': 'Inconsistencia detectada. Llamar a al para confirmar datos', 'recomendacion': 'Verificar en y escribir a para soporte'}
{'id_propiedad': '030-0452-0000', 'nivel_alerta': 'MEDIA', 'motivo_tecnico': 'Posible error de clasificación. Contactar a al', 'recomendacion': 'Solicitar documentación adicional al correo y ajustar registro si aplica'}
{'id_propiedad': '015-0467-0000', 'nivel_alerta': 'BAJA', 'motivo_tecnico': 'Caso atípico por ubicación. Responsable: teléfono', 'recomendacion': 'Revisar historial de pagos y enviar notificación a con pasos a seguir'}
{'id_propiedad': '077-0144-0000', 'nivel_alerta': 'CRITICA', 'motivo_tecnico': 'SIN_DESCRIPCION_TRAS_SANITIZACION', 'recomendacion': 'SIN_DESCRIPCION_TRAS_SANITIZACION'}


# Explicación con LLM

## Cargado de modelo ML

In [5]:
with open("modelo_xgb.pkl", "rb") as f:
    best_pipe = pickle.load(f)

In [6]:
data_path = "../data/"
file_name = "df_pred_test.csv"
file_full_name = "2024_Property_Tax_Roll.csv"
labels = [ 1,  2,  3,  4,  5,  6,  7, 10, 12, 13, 14, 23, 24, 33, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 82, 83, 84]

df_full = pd.read_csv(data_path + file_full_name)
df_final = pd.read_csv(data_path + file_name)
df_final.shape

(8807, 37)

In [7]:
df_final.head()

,P_ID,TAX_MAP,plat,lot,unit,CLASS,SHORT_DESC,LEVY_CODE_1,SHORT_DESC 1,CIVIC,STREET,SUFFIX,FORMATED_ADDRESS,CITY,ZIP_POSTAL,FIRST_NAME,LAST_NAME,COMPANY,FREE_LINE_2,CIVIC 1,STREET 1,S_SUFFIX,UNIT,CITY 1,STATE,ZIP_POSTAL 1,TOTAL_ASSMT,TOTAL_EXEMPT,TOTAL_TAXES,Property_Location,lon,lat,geo_cluster,p_true,pred_class,y_proba_full,nivel_alerta
0,628.0,005-0315-0000,5.0,315.0,0000,13.0,Residential Vacant Land,NO01,NO01,1.0,Steele,St,1 Steele St,Providence,02906,Nathan,Applegate,NaN,70 Evergreen St,70,Evergreen,St,NaN,Providence,RI,02906,73700.0,0.0,1352.40,POINT (-71.403665419 41.846115898),-71.403665,41.846116,3,0.976598,13,"{13: 0.9765976071357727, 23: 0.022168284282088...",BAJA
1,29585.0,077-0417-0000,77.0,417.0,0000,1.0,Single Family,NO01,NO01,29.0,Windmill,St,29 Windmill St,Providence,02904,Liam,Mallon,NaN,29 Windmill St,29,Windmill,St,NaN,Providence,RI,02904,252300.0,0.0,4629.72,POINT (-71.41789503 41.854803004),-71.417895,41.854803,4,0.574853,1,"{1: 0.5748528242111206, 2: 0.42151743173599243...",CRITICA
2,33245.0,086-0452-0000,86.0,452.0,0000,1.0,Single Family,OO01,OO01,124.0,Freeman Pkwy,NaN,124 Freeman Pkwy,Providence,02906,Helen,Burnham,NaN,Helen Metcalf Burnham Revocable Trust,124,Freeman Pkwy,NaN,NaN,Providence,RI,02906,1483700.0,0.0,15518.76,POINT (-71.393158022 41.837744993),-71.393158,41.837745,3,0.986904,1,"{1: 0.9869039058685303, 2: 0.01208311785012483...",BAJA
3,3804.0,013-0212-0000,13.0,212.0,0000,1.0,Single Family,OO01,OO01,67.0,Manning,St,67 Manning St,Providence,02906,David,Ward,NaN,c/o Tiedemann Advisors,601,"Union St, Suite 3315",NaN,NaN,Seattle,WA,98101,1975100.0,0.0,20658.56,POINT (-71.394910007 41.826489019),-71.394910,41.826489,2,0.831954,1,"{1: 0.8319536447525024, 2: 0.10607580095529556...",ALTA
4,21213.0,059-0264-0000,59.0,264.0,0000,1.0,Single Family,OO01,OO01,74.0,Homer,St,74 Homer St,Providence,02905,STEVEN,DACRUZ,NaN,74 HOMER ST,74,HOMER,ST,NaN,PROVIDENCE,RI,02905-1321,269100.0,71992.0,2061.68,POINT (-71.406669985 41.787739984),-71.406670,41.787740,6,0.966079,1,"{1: 0.9660785794258118, 2: 0.03361643105745315...",MEDIA


#### Cálculo de probabilidad de X_test

In [12]:
df_fool = df_final.sample(frac=0.00225)
dict_alerta_info = df_fool.groupby("nivel_alerta")["p_true"].agg(median="median").to_dict()

df_fool.loc[:,"evidencia_registro"] = df_fool.apply(lambda x: build_evidence_packet(x, df_full, dict_alerta_info), axis = 1)

In [24]:
def openai_json_to_dict(text):
    m = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, flags=re.DOTALL | re.IGNORECASE)
    if m:
        text = m.group(1)

    text = text.strip()

    return json.loads(text)
def genera_respuesta(client, dict_info):
    prompt = PROMPT_INSTRUCCIONES.replace("__INFO__TECNICA__", str(dict_info))
    prompt = PROMPT_INSTRUCCIONES.replace("__PROMPT_DETALLE_TECNICO__", PROMPT_DETALLE_TECNICO)
    response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt, config={"temperature": 0.8}).text
    response = openai_json_to_dict(response)
    response["motivo"] = salida_estructurada(response["motivo"])
    response["recomendacion"] = salida_estructurada(response["recomendacion"])

    return response

In [25]:
evidencia_registro = df_fool["evidencia_registro"].values
client = genai.Client(api_key=gemini_key)
respuesta = []
for registro in evidencia_registro:
    print(registro)
    respuesta.append(genera_respuesta(client, registro))




{'id_propiedad': 32938, 'nivel_alerta': 'CRITICA', 'grupos_p_true': {'median': {'ALTA': 0.86311394, 'BAJA': 0.99896324, 'CRITICA': 0.261860075, 'MEDIA': 0.94044365}}, 'info_user': {'full_name': 'Charles Wilcox', 'address': '279-281 Doyle Ave'}, 'modelo': {'class_registrada': 2, 'class_predicha': 1, 'class_pred_top1': 1, 'p_true': 0.22866032, 'p_pred_max': 0.7174351215362549, 'p_margin': 0.48877480153625485, 'top_k': [(1, 0.7174351215362549), (2, 0.22866031527519226), (23, 0.05363382771611214)]}, 'tributario': {'total_assmt': 708000.0, 'total_exempt': 0.0, 'total_taxes': 7405.36, 'tax_rate': 0.01045954802259887, 'exempt_rate': 0.0, 'assmt_pct_in_class_reg': 0.9478876259918507, 'taxes_pct_in_class_reg': 0.8012009435985417, 'exempt_pct_in_class_reg': 0.9066409321609836, 'flags': []}, 'ubicacion': {'zip': '02906', 'geo_cluster': 3}, 'administrativo': {'levy_code_1': 'OO01'}, 'descripcion': {'short_desc': '2 -5 Family'}}


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key expired. Please renew the API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key expired. Please renew the API key.'}]}}

In [14]:
respuesta

[]

In [ ]:
.shape

(40, 38)